# Importando as bibliotecas

In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

# Dataset

In [2]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [3]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_specificity = [] # taxa de verdadeiros negativos ou especificidade
    resultados_f2 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        seed = i
        X_train, X_test, y_train, y_test = procData.holdout(0.2, seed)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação entre o C, gamma e Kernel, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = svm.SVC(random_state = seed)
        grid_svm = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_svm.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        C_best = grid_svm.best_estimator_.C
        gamma_best = grid_svm.best_estimator_.gamma
        kernel_best = grid_svm.best_estimator_.kernel

        SVM = svm.SVC(random_state = seed, C = C_best, gamma = gamma_best, kernel = kernel_best)
        SVM.fit(X_train, y_train)

        #testando o modelo
        y_pred = SVM.predict(X_test)
        cm  = confusion_matrix(y_test, y_pred)
        if exibir_matriz_confusao:
            print(cm)

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        beta = 0.5
        f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_specificity.append(specificity)
        resultados_f2.append(f2_score)

        best_parametros = {"C": C_best, "Gamma": gamma_best, "Kernel": kernel_best};
        resultados_parametros.append(best_parametros)
        


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("Specificity:",specificity)
            print("f2-Score:",f2_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['specificity'] = resultados_specificity
    resultados_gerais['f2'] = resultados_f2
    resultados_gerais['params'] = resultados_parametros
    
    return resultados_gerais

In [4]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\tSpecificity \t|\t fb-Score")
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='  ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['specificity'], axis=0), np.std(dict_metricas['specificity'], axis=0)),end='   ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['f2'], axis=0), np.std(dict_metricas['f2'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [5]:
c_range = [10, 20, 30, 40, 50]
gammas = [0.01, 0.02, 0.03, 0.04, 0.05]
kernels = ["rbf"]
param_grid = {'C': c_range, 'gamma': gammas, 'kernel': kernels}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [6]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('SVM',dict_metricas, rodadas, salvarResultados)

[[23  5]
 [ 3 89]]
Rodada: # 0
{'C': 40, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.9333333333333333
Precision: 0.9468085106382979
Recall: 0.967391304347826
Specificity: 0.8214285714285714
f2-Score: 0.9508547008547008


[[19 14]
 [ 3 84]]
Rodada: # 1
{'C': 10, 'Gamma': 0.01, 'Kernel': 'rbf'}
Accuracy: 0.8583333333333333
Precision: 0.8571428571428571
Recall: 0.9655172413793104
Specificity: 0.5757575757575758
f2-Score: 0.8768267223382046


[[14  8]
 [ 7 91]]
Rodada: # 2
{'C': 40, 'Gamma': 0.04, 'Kernel': 'rbf'}
Accuracy: 0.875
Precision: 0.9191919191919192
Recall: 0.9285714285714286
Specificity: 0.6363636363636364
f2-Score: 0.9210526315789473


[[25  7]
 [ 2 86]]
Rodada: # 3
{'C': 20, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.925
Precision: 0.9247311827956989
Recall: 0.9772727272727273
Specificity: 0.78125
f2-Score: 0.9347826086956521


[[21  7]
 [ 3 89]]
Rodada: # 4
{'C': 50, 'Gamma': 0.04, 'Kernel': 'rbf'}
Accuracy: 0.9166666666666666
Precision: 0.9270833333333334
Recall: 0.96739130

[[24  2]
 [ 5 89]]
Rodada: # 39
{'C': 40, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.9416666666666667
Precision: 0.978021978021978
Recall: 0.9468085106382979
Specificity: 0.9230769230769231
f2-Score: 0.9716157205240176


[[25  6]
 [ 4 85]]
Rodada: # 40
{'C': 10, 'Gamma': 0.04, 'Kernel': 'rbf'}
Accuracy: 0.9166666666666666
Precision: 0.9340659340659341
Recall: 0.9550561797752809
Specificity: 0.8064516129032258
f2-Score: 0.9381898454746138


[[24  8]
 [ 0 88]]
Rodada: # 41
{'C': 10, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.9333333333333333
Precision: 0.9166666666666666
Recall: 1.0
Specificity: 0.75
f2-Score: 0.9322033898305083


[[21  9]
 [ 1 89]]
Rodada: # 42
{'C': 40, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.9166666666666666
Precision: 0.9081632653061225
Recall: 0.9888888888888889
Specificity: 0.7
f2-Score: 0.9232365145228217


[[17  8]
 [ 2 93]]
Rodada: # 43
{'C': 10, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.9166666666666666
Precision: 0.9207920792079208
Recall: 0.9789473684